# Donkey Colab 3.1.0 with GPU(TensorFlow 1.13.1)

Donkey Colab with GPU.
- Version 0.1(2019年10月27日)
- DonkeyCar version 3.1.0(最新版は3.1.1)
- TensorFlow 1.13.1

## 概要
Google Colabを使用し、Donkey Carの学習を行います

上からコードをを実行することで、以下のことができます
- Google Colabの環境設定
- 学習
- 学習結果の取得  




# TensorFlowのインストール

TensorFlowのVersionは、DonkeyCarのRaspberryPi3にはいっているバージョンと同じバージョンをインストールします。  
Google Colabはデフォルトでtensorflow-1.14.0が入っています（2019年9月3日）。  
なので一度アンインストールし、バージョンを揃えています。

In [0]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1

# TensorFlowのバージョン確認

In [0]:
!pip freeze | grep -i -e tensorflow

# GPUの確認

"Found GPU at: /device:GPU:0"と表示されれば、GPUが使用できる状態です。

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# DonkeyCarのインストール

In [0]:
!git clone https://github.com/autorope/donkeycar.git 
%cd donkeycar
!git checkout ba0f7591aff582cee7370c2d9c2962f5cd393987
!pip3 install -e .[pc]

# DonkeyCarのProjectを作成

In [0]:
!donkey createcar --path /content/mycar

# datas.zipのアップロードとデータの解凍
datas.zip（DonkeyCarで集めた学習用データ）をアップロードします。  
下記コードを実行すると、ファイルのアップロード フォームが出現します。  

ファイルは`/content/mycar/data/`以下に送信され、datas.zipの解凍を行っています

In [0]:
import os
from google.colab import files

if(os.path.exists("/content/datas.zip")):
   os.remove("/content/datas.zip")
if(os.path.exists("/content/mycar/data/datas.zip")):
   os.remove("/content/mycar/data/datas.zip")

%cd /content/mycar/data/
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = "/content/mycar/data/" + file_name
print("-----------------------------------------------------")
print("【処理結果】%sにデータをアップロードしました。" % file_path)
print("-----------------------------------------------------")
!unzip -o {file_path}

#学習の開始

Colabの環境構築が出来たので、DonkeyCarの学習を行います。
<br>
<br>
学習の対象は、/content/mycar/data/に存在するフォルダ全部になります。


In [0]:
type = "linear"
trained_mode = "/content/mycar/models/mypilot.h5"

%cd /content/mycar
!python manage.py train --model={trained_mode} --type={type}

# 学習結果の転送
学習結果は、/content/mycar/models/mypilot.h5に保存されます。  下記セルを実行し、H5ファイルをダウンロードします。

RaspberryPiの~/mycar/models/mypilot.h5に学習結果をコピーすることで、DonkeyCarの自動走行ができるようになります。

In [0]:
from google.colab import files

files.download('/content/mycar/models/mypilot.h5')

# 複数の指定タブを学習させる

lsコマンドで、/content/mycar/dataに存在するデータセットのフォルダ名を確認し、フォルダ名を指定して、学習させます。

tub_pathの部分を、自分のフォルダ名に合わせて、複数の指定タブで学習させます。

In [0]:
!ls /content/mycar/data

In [0]:
model_type = "linear" # linear|categorical|rnn|imu|behavior|3d|localizer|latent
trained_mode = "/content/mycar/models/mypilot.h5"
tub_path = "/content/mycar/data/tub_1_19_10_27,/content/mycar/data/tub_2_19_10_27"

%cd /content/mycar
!python manage.py train --model={trained_mode} --type={model_type} --tub={tub_path}